# European Options on Non-Dividend-Paying Stocks
This is the classic case of a European option on a stock with no dividends. The Black-Scholes model
applies, providing a closed-form solution for call and put prices. Implied volatility is obtained 
by inverting the Black-Scholes formula numerically (since no closed-form solution exists for $\sigma$).


In [16]:
# IMPORT NECESSARY PACKAGES
from scipy.stats import norm
from scipy.optimize import brentq
import numpy as np


## Pricing Model
The Black-Scholes (1973) formula uses the Black-Scholes-Merton differential equation to price European calls and puts on a non-dividend-paying underlying. This model assumes no dividends during the option’s life and that exercise can only occur at expiration (European-style). The model requires five inputs:
| Input | Description |
| ----- | ----------- |
| `S` |	Spot price of the underlying asset (i.e. current market price) |
| `K` | Strike price. The fixed price at which the option can be exercised |
| `T` |	Time to expiration, in years (e.g., 6 months = 0.5) |
| `r` |	Risk-free interest rate, continuously compounded (e.g., 5% = 0.05) |
| `sigma` |	Volatility of the underlying asset (annualized standard deviation) |
| `option_type` | Type of option: `c` for call, `p` for put |

The formulas are:
$$
C=SN(d_1)-Ke^{-rT}N(d_2)
$$
$$
P=Ke^{-rT}N(-d_2)-SN(-d_1)
$$
With:
$$
d_{1,2} = \frac{\ln\left(\frac{S}{K}\right)+T\left(r\pm \frac{\sigma ^2}{2}\right)}{\sigma \sqrt{T}}
$$
And $N(\cdot)$ as the standard normal cumulative distribution function. The put price is obtained via put-call parity. 

#### Function Signature
Returns the theoretical option price for a call or put:

`bs_price(S, K, T, r, sigma, option_type) -> float`

In [24]:
def get_ds(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (T*(r+((sigma**2)/2)))) / (sigma * np.sqrt(T))
    d2= d1 - (sigma * np.sqrt(T))
    return (d1, d2)

def bs_price(S, K, T, r, sigma, option_type):
    d1 = get_ds(S, K, T, r, sigma)[0]
    d2 = get_ds(S, K, T, r, sigma)[1]
    match option_type:
        case 'c':
            return (S * norm.cdf(d1)) - (K * np.exp(-1*r*T) * norm.cdf(d2))
        case 'p':
            return (K * np.exp(-1*r*T) * norm.cdf(-1*d2)) - (S * norm.cdf(-1*d1))

## Implied Volatility

Black-Scholes implied volatility – Solve for $\sigma$ by numerically inverting the Black-Scholes pricing function. Given a market option price, this function finds the $\sigma$ that reproduces this price in the Black-Scholes formula. There is no closed-form solution for implied vol in the Black-Scholes model, so the Brent's (iterative) method is used.

#### Function Signature
Returns the implied volatility for given option:

`bs_implied_vol(price, S, K, T, r, option_type) -> float`

In [20]:
def bs_implied_vol(price, S, K, T, r, option_type):
    def objective(sigma):
        return bs_price(S, K, T, r, sigma, option_type) - price
    return brentq(objective, 1e-6, 5.0)

## Greeks

These are all first or second-order partial derivatives of the option pricing function $V(S,K,T,r,\sigma)$ to measure how the option's price reacts to changes in market inputs.

- **Delta ($\Delta$):** Sensitivity of the option’s price to changes in the underlying asset price.
- **Gamma ($\Gamma$):** The rate of change of delta with respect to the underlying price (i.e. how stable delta is).
- **Vega ($\nu$):** Sensitivity of the option’s price to changes in volatility ($\sigma$).
- **Theta ($\Theta$):** Sensitivity of the option’s price to the passage of time (i.e., time decay).
- **Rho ($\rho$):** Sensitivity of the option’s price to changes in the risk-free interest rate.

These measures are defined as follows:
$$\Delta = \frac{\partial V}{\partial S}$$
$$\Gamma = \frac{\partial^2 V}{\partial S^2}$$
$$\nu = \frac{\partial V}{\partial \sigma}$$
$$\Theta = -\frac{\partial V}{\partial T}$$
$$\rho = \frac{\partial V}{\partial r}$$

The only new variable introduced in these calculations is $n(\cdot)$, which is the standard normal probability density function.
#### Function Signature
Returns a dictionary of the greeks for the given option:

`bs_implied_vol(price, S, K, T, r, option_type) -> dict`

`{"delta": , "gamma": , "vega": , "theta": , "rho": ,}`

In [ ]:
def bs_greeks(S, K, T, r, sigma, option_type):
    sol = {"delta":None, "gamma":None, "vega":None, "theta":None, "rho":None}
    d1 = get_ds(S, K, T, r, sigma)[0]
    d2 = get_ds(S, K, T, r, sigma)[1]
    
    match option_type:
        case 'c':
            delta = norm.cdf(d1)
            gamma = norm.pdf(d1)/(S * sigma * np.sqrt(T))
            vega = S * norm.pdf(d1) * np.sqrt(T)
            theta = (-1 * ((S * norm.pdf(d1) * sigma)/(2 * np.sqrt(T)))) - (r * K * np.exp(-1*r*T) * norm.cdf(d2))
            rho = K * T * np.exp(-1*r*T) * norm.cdf(d2)
        case 'p':
            delta = norm.cdf(d1) - 1
            gamma = norm.pdf(d1)/(S * sigma * np.sqrt(T))
            vega = S * norm.pdf(d1) * np.sqrt(T)
            theta = (-1 * ((S * norm.pdf(d1) * sigma)/(2 * np.sqrt(T)))) + (r * K * np.exp(-1*r*T) * norm.cdf(-1*d2))
            rho = -1 * K * T * np.exp(-1*r*T) * norm.cdf(-1*d2)

    sol["delta"] = delta
    sol["gamma"] = gamma
    sol["vega"] = vega / 100
    sol["theta"] = theta / 365
    sol["rho"] = rho / 100
    
    return sol